# Working with Databases

So far, we've worked exclusively with data that could conveniently fit in memory. Generally speaking, data sets of under 20 megabytes or so can be loaded easily on most modern laptops, although subsequent analysis activities may be slow. Larger data sets can be noticeably slow to even load into memory. For example, you may  have noticed that so far, I've been using a small subset of the NOAA-GHCN data corresponding to the decade 2011-2020. There's much  more data, and I have the complete data set in a file (also uploaded as a `.zip` archive on CCLE) in the same directory as these lecture notes on my personal machine. The complete set of data is not too big to fit in memory, but it's large enough that it's noticeably slow to even load into memory. Let's take a look. 

In [1]:
import pandas as pd
%timeit df = pd.read_csv("temps.csv")

1 loop, best of 5: 1.69 s per loop


Two seconds just to load the data set! Note: I'm not even *down*loading the data set -- this is just how long it takes to move the data from my computer's storage into RAM. 

We can get a pretty good estimate of how much RAM is required to store the data using a handy method of Pandas data frames: 

In [2]:
df = pd.read_csv("temps.csv")
df.shape

(1359937, 14)

In [3]:
df.memory_usage().sum() / 10**6 # rough size in megabytes

152.313072

The data contains 1.35M rows and 14 columns. Altogether, these consume roughly 150 MB of RAM. This is an appreciable fraction of the RAM on most laptops. In some cases, we might need to work with even larger data sets that don't fit in RAM at all. 

## Introducing Databases

When dealing with large data sets like these, it's relatively rare that we absolutely *have* to operate on the entire data set. In most cases, we can work with parts of the data at a time. In the context of the temperature data, 

1. We might want temperature measurements only between years 1990-2020. 
2. We might want temperature measurements only for a certain set of countries - maybe in Asia, say. 
3. We might only want temperature measurements only in the month of March. 
4. We might want a random 1% of all the data. 

**Databases** provide us with a structured way to move subsets of data from storage into memory. Python has a module called `sqlite3` (already installed) which we can use to create, manipulate, and query databases. There's also a very handy `pandas` interface, enabling us to efficiently create `pandas` data frames containing exactly the data that we want. Let's get started. 

### Creating and Populating Databases

In [4]:
import sqlite3

In [5]:
conn = sqlite3.connect("temps.db") # create a database in current directory called temps.db

A quick check in our file directory reveals that we now have a file called `temps.db`. Great! 

There are many ways to add data to a database. Since we are already familiar with Pandas data frames, we'll make use of some extremely convenient functionality which allows us to directly write data frames to a database. 

But wait! Wasn't the whole point of this to avoid reading in an entire data frame? Indeed! Pandas supplies a nice approach to this using the familiar `pd.read_csv()` function. Supplying a value of `chunksize` will cause `read_csv()` to return not a data frame but an *iterator*, each of whose elements is a piece of the data with number of rows equal to `chunksize`. The data is read "on the fly" -- i.e. it's not actually read until we start querying the iterator. Here's an example: 

In [6]:
df_iter = pd.read_csv("temps.csv", chunksize = 100000)

In [7]:
df = df_iter.__next__()

In [8]:
df.head()

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,ACW00011604,1961,-89.0,236.0,472.0,773.0,1128.0,1599.0,1570.0,1481.0,1413.0,1174.0,510.0,-39.0
1,ACW00011604,1962,113.0,85.0,-154.0,635.0,908.0,1381.0,1510.0,1393.0,1163.0,994.0,323.0,-126.0
2,ACW00011604,1963,-713.0,-553.0,-99.0,541.0,1224.0,1627.0,1620.0,1596.0,1332.0,940.0,566.0,-108.0
3,ACW00011604,1964,62.0,-85.0,55.0,738.0,1219.0,1442.0,1506.0,1557.0,1221.0,788.0,546.0,112.0
4,ACW00011604,1965,44.0,-105.0,38.0,590.0,987.0,1500.0,1487.0,1477.0,1377.0,974.0,31.0,-178.0


Great! Actually, there are a few cleaning steps that we'll make before incorporating this data into our database. If you're not sure what's going on here, you may want to check the previous lecture in which we cover `stack`.  

In [9]:
def prepare_df(df):
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)

In [10]:
df = prepare_df(df)
df.head()

,ID,Year,Month,Temp
0,ACW00011604,1961,1,-0.89
1,ACW00011604,1961,2,2.36
2,ACW00011604,1961,3,4.72
3,ACW00011604,1961,4,7.73
4,ACW00011604,1961,5,11.28


Ok! We are finally ready to populate a *table* in our database. 

You can think of a table as a data frame-like object, represented in SQLite rather than in Python. The `df.to_sql()` method writes to a specified table in the database (the `conn` object from earlier). We need to specify `if_exists` to ensure that we add each piece to the table, rather than overwriting them each time. 

In [11]:
import pandas as pd
df_iter = pd.read_csv("temps.csv", chunksize = 100000)
for df in df_iter:
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "append", index = False)

Let's similarly add a table for the metadata in our database. This is a pretty small data set so we don't need to worry about reading it in by chunks. 

In [12]:
stations = pd.read_csv("station-metadata.csv")
stations.to_sql("stations", conn, if_exists = "replace", index = False)

Now we have a database containing two tables. Let's just check that this is indeed the case. 

In [13]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('temperatures',), ('stations',)]


Ok wait -- there's actually a lot that just happened here! 

First: the *cursor* is our primary way to interact with the database. The cursor `execute`s *SQL commands*. SQL is actually its own mini-programming language specifically designed for interacting with databases. We do need to learn a bit of SQL in order to work with databases, but I promise it won't be too bad. 

Now let's take a look at the actual command that we `execute`d.

```sql
SELECT name FROM sqlite_master WHERE type='table'
```

Let's break this down: 

- `SELECT name`: Show me the entries in the `name` column
- `FROM sqlite_master`: of the `sqlite_master` table
- `WHERE type='table'`: subject to the condition that the entry in the `type` column of `sqlite_master` is equal to ``table`` (i.e. don't include other kinds of objects)

Finally, `cursor.fetchall()` returns the a list containing all the items returned by the query, which we then print. 

Let's get more detailed information about the items in each table. For example, we can inspect the column names and data types in each. This is a good way to check that we actually populated our database correctly. 

In [14]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)


This looks pretty good! We have two tables, called `temperatures` and `stations`. The column names are what we would expect. Notice that `sql` has automatically inferred the data types, such as `REAL` and `TEXT`, from the input. Handy! 

#### Basic Queries

Now we're ready to perform some basic queries. 

Which stations have records for the year 1990? In `pandas` we would do this like: 

```
temperatures["id"][temperatures["year"] == 1990]
```

The SQL syntax is very different, but contains all the same ideas. 

- `SELECT`, like the syntax `temperatures["id"]`, controls which column(s) will be returned. 
- `FROM` tells us which table to return columns from. 
- `WHERE` is like the Boolean index `[temperatures["year"] == 1990]`. Only rows in which this criterion is satisfied will be returned. 

**NOTE**: SQL commands are case-insensitive, but it's considered good practice to place SQL keywords in ALL CAPS while names of columns and other content goes in lowercase. 

In [15]:
cursor.execute("SELECT id FROM temperatures WHERE year=1990;")
result = cursor.fetchall() # get all results
result[0:10]

[('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',),
 ('ACW00011604',)]

Note that the result is returned as a tuple. This isn't very useful when we are just querying a single column, but we can also get multiple columns, in which case the tuple representation makes a bit more sense: 

In [16]:
cursor.execute("SELECT id, temp FROM temperatures WHERE year=1990;")
result = [cursor.fetchone() for i in range(10)]  # get just the first 10 results
result

[('ACW00011604', 4.51),
 ('ACW00011604', 5.91),
 ('ACW00011604', 6.01),
 ('ACW00011604', 8.11),
 ('ACW00011604', 12.31),
 ('ACW00011604', 15.81),
 ('ACW00011604', 16.71),
 ('ACW00011604', 17.11),
 ('ACW00011604', 12.11),
 ('ACW00011604', 9.51)]

We can include multiple criteria in `WHERE` using the Boolean operators `AND` and `OR`. 

In [17]:
cursor.execute("SELECT id, temp FROM temperatures WHERE year=1990 AND month=1;")
result = [cursor.fetchone() for i in range(10)]  # get just the first 10 results
result

[('ACW00011604', 4.51),
 ('AE000041196', 17.78),
 ('AEM00041184', 18.3),
 ('AEM00041194', 18.87),
 ('AEM00041216', 18.91),
 ('AEM00041217', 18.57),
 ('AFM00040911', 4.4),
 ('AFM00040948', 0.16),
 ('AFXLT299877', -5.6),
 ('AG000060390', 10.74)]

Which stations are either far south or far north? 

In [18]:
cursor.execute("SELECT id FROM stations WHERE latitude>80 OR latitude<-80;")
result = [cursor.fetchone() for i in range(10)]  # get just the first 10 results
result

[('AYM00089013',),
 ('AYM00089108',),
 ('AYM00089314',),
 ('AYM00089329',),
 ('AYM00089332',),
 ('AYM00089345',),
 ('AYM00089377',),
 ('AYM00089577',),
 ('AYM00089799',),
 ('AYM00089873',)]

## Relations

What if we wanted the temperature measurements taken at elevations higher than 1000m above sea-level? This involves information from both the `temperature` table and the `stations` table. In order to perform this kind of query, we need to first ensure that these tables are appropriately *related*. 

We know from our previous interactions with this data set that the `id` column in the `stations` table corresponds to the `id` column in the `temperature` table. We'd like to incorporate this relationship in our database. From here, the syntax gets only slightly more complicated. Here's what we'll do: 

1. First, we'll give the `temperatures` and `stations` tables aliases `T` and `S` just to make the command more readable. We can also split the command onto multiple lines. 
2. Second, we need to add an `LEFT JOIN` call which indicates which column of `T` corresponds to which column of `S`. This is similar to `pd.merge` from a few lectures ago. There are also `RIGHT JOINT`, `INNER JOIN` and `OUTER JOIN` operators that control how duplicates and mismatched rows are handled. 



In [19]:
cmd = \
"""
SELECT T.id, T.month, T.temp
FROM temperatures T
LEFT JOIN stations S ON T.id = S.id
WHERE S.latitude>80 OR S.latitude<-80
"""

cursor.execute(cmd)
result = [cursor.fetchone() for i in range(10)]  # get just the first 10 results
result

[('AYM00089013', 4, -41.08),
 ('AYM00089013', 5, -48.4),
 ('AYM00089013', 6, -50.7),
 ('AYM00089013', 7, -49.14),
 ('AYM00089013', 10, -43.66),
 ('AYM00089013', 11, -30.94),
 ('AYM00089013', 12, -20.55),
 ('AYM00089013', 1, -19.54),
 ('AYM00089013', 2, -31.39),
 ('AYM00089013', 3, -39.56)]

The cursor is very good for prototyping your SQL queries, but once you've gotten the hang of it, the easiest approach is to use another convenience function from Pandas: 

In [20]:
df = pd.read_sql_query(cmd, conn)

You can actually obtain columns from both tables this way: 

In [21]:
cmd = \
"""
SELECT T.id, T.month, T.temp, S.latitude, S.longitude, S.name
FROM temperatures T
LEFT JOIN stations S ON T.id = S.id
WHERE S.latitude>80 OR S.latitude<-80
"""

df = pd.read_sql_query(cmd, conn)

It's good practice to close your database connection once you're done using it. 

In [22]:
conn.close()

Next time you need to pull some data from your database, just reopen the connection! 

#### *Flashforward -- Sometime in The Future*

In [23]:
import sqlite3
conn = sqlite3.connect("temps.db")

In [24]:
cmd = \
"""
SELECT T.id, T.month, T.temp, S.latitude, S.longitude, S.name
FROM temperatures T
LEFT JOIN stations S ON T.id = S.id
WHERE S.latitude>80 OR S.latitude<-80
"""

df = pd.read_sql_query(cmd, conn)
conn.close()

# the rest of my analysis...